In [1]:
import serial
import struct
import numpy as np
import time
import signal 
import glob
import timeit
from time import time, sleep

savePointCloud = True
printPointCloud = True

configPort_name = '/dev/tty.SLAB_USBtoUART'
dataPort_name = '/dev/tty.SLAB_USBtoUART3'
configFile_name = '/Users/gbromley/code/ODIN/chirp_3DPeople.cfg'
pointCloud_fileName = './output_files/PointCloud.txt'

syncPattern = 0x708050603040102

# 0x0102, 0x0304, 0x0506, 0x0708


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:


savePointCloud = True
saveTracks = True
saveTracksID = True

configPort_name = '/dev/tty.SLAB_USBtoUART'
dataPort_name = '/dev/tty.SLAB_USBtoUART3'
configFile_name = '/Users/gbromley/code/ODIN/config/chirp_3DPeople.cfg'
pointCloud_fileName = './output_files/PointCloud.txt'
targetsObject_fileName = './output_files/Targets.txt'
targetsIndex_FileName = './output_files/TargetsIndex.txt'

syncPattern = 0x708050603040102

try:

	print('Reading configuration file')
	with open(configFile_name, 'r') as fp:
		cnt = 0
		commands = []
		for line in fp:
			if (len(line) > 1):
				if (line[0] != '%'):
					commands.append(line)
					cnt += 1
	fp.close()

	print('Opening ports')
	configPort = serial.Serial(configPort_name, 115200, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0.3)
	dataPort = serial.Serial(dataPort_name, 921600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0.3)
	dataPort.reset_output_buffer()
	print(dataPort.is_open)
	print(configPort.is_open)
	for i in range(cnt):
		configPort.write(bytearray(commands[i].encode()))
		sleep(20e-3)

	packetHeader = bytearray([])

	tlvHeaderLen = 8
	headerLen = 48

	while(True):

		timePacket = time()
		print(time())
		dataPort.reset_output_buffer()
		packetHeader += dataPort.read(headerLen-len(packetHeader))
		print(packetHeader)
		print(type(struct.unpack('Q9I2H', packetHeader[:headerLen])))
		sync, version, totalPacketLen, platform, frameNumber, subFrameNumber, chirpProcessingMargin,frameProcessingMargin, trackProcessTime, uartSentTime, numTLVs, checksum =  struct.unpack('Q9I2H', packetHeader[:headerLen])
		print(numTLVs)
		if (sync == syncPattern):

			packetHeader = bytearray([])
			packetPayload = dataPort.read(totalPacketLen-headerLen)

			for i in range(numTLVs):

				tlvType, tlvLength = struct.unpack('2I', packetPayload[:tlvHeaderLen])
				
				if (tlvType > 20 or tlvLength > 10000):
					packetHeader = bytearray([])
					break

				packetPayload = packetPayload[tlvHeaderLen:]

				if (tlvType == 6):

					pointUnit = struct.unpack('5f', packetPayload[:20])
					packetPayload = packetPayload[20:]
					numDetectedObj = int((tlvLength-tlvHeaderLen-20)/8)
					detectedObjects = np.zeros((numDetectedObj, 5))
					results_string = ''

					print('numDetectedObj: %d' % numDetectedObj)

					for j in range(numDetectedObj):

						elevation_j, azimuth_j, doppler_j, range_j, snr_j = struct.unpack('2bh2H', packetPayload[:8])

						detectedObjects[j, 0] = range_j * pointUnit[3]
						detectedObjects[j, 1] = azimuth_j * pointUnit[1] * 180/np.pi
						detectedObjects[j, 2] = elevation_j * pointUnit[0] * 180/np.pi
						detectedObjects[j, 3] = doppler_j * pointUnit[2]
						detectedObjects[j, 4] = snr_j * pointUnit[4]

						packetPayload = packetPayload[8:]

						results_string += '%1.3f %1.3f %1.3f %1.3f %1.3f ' % (detectedObjects[j, 0], detectedObjects[j, 1], detectedObjects[j, 2], detectedObjects[j, 3], detectedObjects[j, 4])

					if (savePointCloud):
						filePoints = open(pointCloud_fileName, 'a')
						filePoints.write(results_string + '%1.3f\n' % timePacket)
						filePoints.close()

				elif (tlvType == 7):

					numOfTargets = int((tlvLength-tlvHeaderLen)/112)
					DetectedTargets = np.zeros((numOfTargets, 10))
					results_string = ''

					print('numOfTargets: %d' % numOfTargets)

					for j in range(numOfTargets):

						tid_j, posX_j, posY_j, posZ_j, velX_j, velY_j, velZ_j, accX_j, accY_j, accZ_j = struct.unpack('I9f', packetPayload[:40])
						ec = struct.unpack('16f', packetPayload[40:40+64])
						g, confidenceLevel = struct.unpack('2f', packetPayload[40+64:112])

						DetectedTargets[j, 0] = tid_j
						DetectedTargets[j, 1] = posX_j
						DetectedTargets[j, 2] = posY_j
						DetectedTargets[j, 3] = posZ_j
						DetectedTargets[j, 4] = velX_j
						DetectedTargets[j, 5] = velY_j
						DetectedTargets[j, 6] = velZ_j
						DetectedTargets[j, 7] = accX_j
						DetectedTargets[j, 8] = accY_j
						DetectedTargets[j, 9] = accZ_j

						results_string += '%d %1.3f %1.3f %1.3f %1.3f %1.3f %1.3f %1.3f %1.3f %1.3f ' % (DetectedTargets[j, 0], DetectedTargets[j, 1], DetectedTargets[j, 2], DetectedTargets[j, 3], DetectedTargets[j, 4], DetectedTargets[j, 5], DetectedTargets[j, 6], DetectedTargets[j, 7], DetectedTargets[j, 8], DetectedTargets[j, 9])

						packetPayload = packetPayload[112:]
						print(results_string)
					if (saveTracks):
						fileTracks = open(targetsObject_fileName, 'a')
						fileTracks.write(results_string + '%1.3f\n' % timePacket)
						fileTracks.close()

				elif (tlvType == 8):

					numDetectedObj_previous = tlvLength-tlvHeaderLen
					targetIndex = [0]*numDetectedObj_previous
					results_string = ''

					for j in range(numDetectedObj_previous):

						targetIndex[j] = struct.unpack('B', packetPayload[:1])
						results_string += '%d ' % targetIndex[j]
						packetPayload = packetPayload[1:]

					if (saveTracksID):
						fileTracksID = open(targetsIndex_FileName, 'a')
						fileTracksID.write(results_string + '%1.3f\n' % timePacket)
						fileTracksID.close()

			if (numTLVs > 0):
				print(' ')

		else:
			packetHeader = packetHeader[1:]

except:
    dataPort.close()
    configPort.close()
    raise






Reading configuration file
Opening ports
True
True
1719602138.296428
bytearray(b'\x02\x01\x04\x03\x06\x05\x08\x07\x04\x00\x05\x03\x14\x01\x00\x00Ch\n\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00!$\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00^_')
<class 'tuple'>
1
numDetectedObj: 25


FileNotFoundError: [Errno 2] No such file or directory: './output_files/PointCloud.txt'

In [6]:
dataPort.read(headerLen-len(packetHeader))

PortNotOpenError: Attempting to use a port that is not open

In [5]:
def readConfigFile(configFileName):
    try:
        with open(configFileName, 'r') as fp:
            cnt = 0
            commands = []
            for line in fp:
                if (len(line) > 1):
                    if (line[0] != '%'):
                        commands.append(line)
                        cnt += 1
        return (cnt,commands)
    except FileNotFoundError as err:
        print("Chirp config file not found.")
    finally:
        fp.close()
        


In [6]:
readConfigFile(configFile_name)

(30,
 ['sensorStop\n',
  'flushCfg\n',
  'dfeDataOutputMode 1\n',
  'channelCfg 15 7 0\n',
  'adcCfg 2 1\n',
  'adcbufCfg -1 0 1 1 1\n',
  'lowPower 0 0\n',
  'profileCfg 0 60.75 30.00 25.00 59.10 394758 0 54.71 1 96 2950.00 2 1 36 \n',
  'chirpCfg 0 0 0 0 0 0 0 1\n',
  'chirpCfg 1 1 0 0 0 0 0 2\n',
  'chirpCfg 2 2 0 0 0 0 0 4\n',
  'frameCfg 0 2 96 0 55 1 0\n',
  'dynamicRACfarCfg -1 4 4 2 2 8 12 4 8 5.00 8.00 0.40 1 1\n',
  'staticRACfarCfg -1 6 2 2 2 8 8 6 4 8.00 15.00 0.30 0 0\n',
  'dynamicRangeAngleCfg -1 0.75 0.0010 1 0\n',
  'dynamic2DAngleCfg -1 1.5 0.0300 1 0 1 0.30 0.85 8.00\n',
  'staticRangeAngleCfg -1 0 8 8\n',
  'antGeometry0 -1 -1 0 0 -3 -3 -2 -2 -1 -1 0 0\n',
  'antGeometry1 -1 0 -1 0 -3 -2 -3 -2 -3 -2 -3 -2\n',
  'antPhaseRot 1 -1 1 -1 1 -1 1 -1 1 -1 1 -1\n',
  'fovCfg -1 70.0 20.0\n',
  'compRangeBiasAndRxChanPhase 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0\n',
  'staticBoundaryBox -3 3 1 3.5 -2 2\n',
  'boundaryBox -3.5 3.5 0.5 4 -2.5 2.5\n',
  'sensorPositio

In [7]:
serialPorts = glob.glob('/dev/tty.SLAB*')
print(serialPorts)

['/dev/tty.SLAB_USBtoUART', '/dev/tty.SLAB_USBtoUART3']


In [10]:

configPort = serial.Serial(serialPorts[1], 115200, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0.3)
dataPort = serial.Serial(serialPorts[0],921600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0.3)

In [8]:
configPort.close()
dataPort.close()

In [11]:
def writeConfig(configPort, configFileName):
    cnt,commands = readConfigFile(configFileName)
    
    for i in range(cnt):
        configPort.write(bytearray(commands[i].encode()))
        time.sleep(20e-3)
        response = bytearray([])
        while(configPort.in_waiting > 0):
            response += configPort.read(1)
        print(response.decode())

In [12]:
writeConfig(configPort, configFile_name)

sensorStop
Ignored: Sensor is already stopped
Done
mmwDemo:/>
flushCfg
Done
mmwDemo:/>
dfeDataOutputMode 1
Done
mmwDemo:/>
channelCfg 15 7 0
Done
mmwDemo:/>
adcCfg 2 1
Done
mmwDemo:/>
adcbufCfg -1 0 1 1 1
Done
mmwDemo:/>
lowPower 0 0
Done
mmwDemo:/>
profileCfg 0 60.75 30.00 25.00 59.10 394758 0 54.71 1 96 2950.00 2 1 36 
Done
mmwDemo:/>
chirpCfg 0 0 0 0 0 0 0 1
Done
mmwDemo:/>
chirpCfg 1 1 0 0 0 0 0 2
Done
mmwDemo:/>
chirpCfg 2 2 0 0 0 0 0 4
Done
mmwDemo:/>
frameCfg 0 2 96 0 55 1 0
Done
mmwDemo:/>
dynamicRACfarCfg -1 4 4 2 2 8 12 4 8 5.00 8.00 0.40 1 1
Done
mmwDemo:/>
staticRACfarCfg -1 6 2 2 2 8 8 6 4 8.00 15.00 0.30 0 0
Done
mmwDemo:/>
dynamicRangeAngleCfg -1 0.75 0.0010 1 0
Done
mmwDemo:/>
dynamic2DAngleCfg -1 1.5 0.0300 1 0 1 0.30 0.85 8.00
Done
mmwDemo:/>
staticRangeAngleCfg -1 0 8 8
Done
mmwDemo:/>
antGeometry0 -1 -1 0 0 -3 -3 -2 -2 -1 -1 0 0
Done
mmwDemo:/>
antGeometry1 -1 0 -1 0 -3 -2 -3 -2 -3 -2 -3 -2
Done
mmwDemo:/>
antPhaseRot 1 -1 1 -1 1 -1 1 -1 1 -1 1 -1
Done
mmwDemo:/>
fo

In [9]:
def signal_handler(signal, frame):
        global run
        print("exiting")
        run = False

In [11]:
dataPort.isOpen()

True

In [11]:
signal.signal(signal.SIGINT, signal_handler)   

<function _signal.default_int_handler>

In [169]:
dataPort.in_waiting


0

In [13]:
def readPacketHeader(dataPort):
    dataPort.reset_input_buffer()

    
    
    syncPattern = 0x708050603040102
    ### Packet header is 40 total bytes, 8 bytes for magic/sync word and 32 bytes for 8 other variables. 
    ### The format for struct.unpack is Q8I because the magic word is 8 bytes and Q stands for unsigned long long. 8I means that 8 unsigned ints follow
    packetHeader = bytearray([])
    tlvHeaderLen = 8
    headerLen = 40
    timePacket = time.time()
    run = True
    
    while(True):
        
        try:
            
            
            ### These lines of code basically grab the header data and shifts it by one until the sync value is found. 
            packetHeader += dataPort.read(headerLen-len(packetHeader))
            
            #print(len(packetHeader))
            sync, version, totalPacketLen, platform, frameNumber, timeCpuCycles, numDetectedObj, numTLVs, subFrameNumber =  struct.unpack('Q8I', packetHeader[:headerLen])
            if (sync == syncPattern):
                
                print('success')
                print(time.perf_counter())
                print(f'MagicWord: {sync}, Version: {version}, Total Packet Length: {totalPacketLen}, Platform: {platform}, Frame Number: {frameNumber}, Time CPU Cycles {timeCpuCycles}, Number of Objects: {numDetectedObj}, Number of TLVs: {numTLVs}, Sub Frame Number: {subFrameNumber}')
                    
                return sync, version, totalPacketLen, platform, frameNumber, timeCpuCycles, numDetectedObj, numTLVs, subFrameNumber
                #break
            else:
                packetHeader = packetHeader[1:]
        except serial.SerialException as serialError:
            print('Something is not correct with the serial port')
            break
            
        except KeyboardInterrupt:
            break

In [14]:


readPacketHeader(dataPort)


success
715006.065415167
MagicWord: 506660481457717506, Version: 50659332, Total Packet Length: 84, Platform: 682051, Frame Number: 15, Time CPU Cycles 0, Number of Objects: 0, Number of TLVs: 7435, Sub Frame Number: 5


(506660481457717506, 50659332, 84, 682051, 15, 0, 0, 7435, 5)

In [170]:
dataPort.reset_input_buffer()
dataPort.in_waiting


0

In [173]:
def testData():
    
    sync, version, totalPacketLen, platform, frameNumber, timeCpuCycles, numDetectedObj, numTLVs, subFrameNumber = readPacketHeader(dataPort)
    print(sync)
    

In [174]:
dataPort.read()

b'\x00'

In [15]:
def readDataFrame(dataPort):
    start = time.time()
    timePacket = time.time()
    headerLen = 40
    tlvHeaderLen = 8
    radarData = []
    #packetHeader += dataPort.read(headerLen-len(packetHeader))
    ### Packet header is 40 total bytes, 8 bytes for magic/sync word and 32 bytes for 8 other variables. 
    ### The format for struct.unpack is Q8I because the magic word is 8 bytes and Q stands for unsigned long long. 8I means that 8 unsigned ints follow
    sync, version, totalPacketLen, platform, frameNumber, timeCpuCycles, numDetectedObj, numTLVs, subFrameNumber = readPacketHeader(dataPort)

    if (sync == syncPattern):

        packetPayload = bytearray([])
        packetPayload = dataPort.read(totalPacketLen-headerLen)

        detectedObjects = np.zeros((numDetectedObj, 6))

        for i in range(numTLVs):

            tlvType, tlvLength = struct.unpack('2I', packetPayload[:tlvHeaderLen])

            if (tlvType > 20 or tlvLength > 10000):
                packetHeader = bytearray([])
                break

            packetPayload = packetPayload[tlvHeaderLen:]

            if (tlvType == 1):

                for j in range(numDetectedObj):

                    x, y, z, v = struct.unpack('4f', packetPayload[:16])
                    
                    detectedObjects[j, 0] = x
                    detectedObjects[j, 1] = y
                    detectedObjects[j, 2] = z
                    detectedObjects[j, 3] = v

                    packetPayload = packetPayload[16:]

            elif (tlvType == 7):

                for j in range(numDetectedObj):

                    snr, noise = struct.unpack('2H', packetPayload[:4])

                    detectedObjects[j, 4] = snr
                    detectedObjects[j, 5] = noise

                    #if (printPointCloud):
                        
                        #print('x: %1.3f m, y: %1.3f m, z: %1.3f m, v: %1.3f m/s, snr: %d, noise: %d' % (detectedObjects[j, 0], detectedObjects[j, 1], detectedObjects[j, 2], detectedObjects[j, 3], detectedObjects[j, 4], detectedObjects[j, 5]))

                    packetPayload = packetPayload[4:]
    return detectedObjects

In [16]:
readDataFrame(dataPort)

success
715032.796155125
MagicWord: 506660481457717506, Version: 50659332, Total Packet Length: 84, Platform: 682051, Frame Number: 29, Time CPU Cycles 0, Number of Objects: 0, Number of TLVs: 7427, Sub Frame Number: 5


array([], shape=(0, 6), dtype=float64)

In [194]:
from IPython.display import clear_output
while(True):
    clear_output(wait=True)
    readDataFrame(dataPort)

TypeError: cannot unpack non-iterable NoneType object

In [175]:
readDataFrame(dataPort)

TypeError: cannot unpack non-iterable NoneType object